# 使用pytorch实现mnist数据集

In [ ]:
import os
import struct
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
from matplotlib import pyplot as plt

[mnist 数据集官网](http://yann.lecun.com/exdb/mnist/)

In [ ]:
def read_mnist():
    """
    官方下载数据集，然后制作数据
    需要测试集请前往官网
    """
    with open("train/"+"train-labels.idx1-ubyte","rb") as f:
        magic, n = struct.unpack(">II",f.read(8))
        labels = np.fromfile(f,dtype=np.uint8).reshape(n).astype("int64")

    with open("train/"+"train-images.idx3-ubyte","rb") as f:
        magic, images_num, rows, cols = struct.unpack(">IIII",f.read(16))
        images = np.fromfile(f,dtype=np.uint8).reshape(images_num, 1, rows, cols).astype('float32')
    return labels, images

In [ ]:
from torch.utils.data import Dataset,DataLoader

labels, images = read_mnist()
batch_size = 32
class dataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.len = len(labels)
    def __getitem__(self, index):
        return torch.tensor(self.images[index]), torch.tensor(self.labels[index])
    def __len__(self):
        return self.len
    
ratio = 0.7
offset = int(ratio*len(labels))
train_dataset = dataset(images[:offset], labels[:offset])
train_data = DataLoader(train_dataset, batch_size, shuffle=True)

In [ ]:
# 使用卷积神经网络来完成识别
"""
非常简单的卷积网络，没有池化等操作
"""
class neural_network(nn.Module):
    def __init__(self, rows, cols, num_class):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 100, 3) # 20
        self.conv2 = nn.Conv2d(100, 200, 3) # 12
        self.fc = nn.Linear(24,100) # pytorch 的全连接层 in_feature, out_feature
        self.pre = nn.Linear(100*24*200, num_class)
    
    def forward(self, img):
        conv1 = self.conv1(img)
        conv2 = self.conv2(conv1)
        fc = self.fc(conv2)
        fc = fc.view(-1, 100*24*200)
        predict = self.pre(fc)
        return predict

In [ ]:
import torch.nn.functional as F

def inference_program(predict, label):
    loss_func = F.cross_entropy
    
    def accuracy(x,y):
        x_arg = x.argmax(dim=1)
        return (x_arg==y).float().sum()/(len(y))
    
    cost = loss_func(predict, label)
    acc = accuracy(predict, label)
    return cost, acc

In [ ]:
from torch.autograd import Variable

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = neural_network(28,28,10)
net = net.to(device)
opt = torch.optim.Adam(net.parameters(), lr=0.001)
print(net)

for epoch in range(10):
    for step_id, data in enumerate(train_data):
        var_x, label = Variable(data[0]), Variable(data[1])
        var_x = var_x.to(device)
        label = label.to(device)
        predict = net(var_x)
        cost, acc = inference_program(predict, label)
        cost.backward()
        opt.step()
        opt.zero_grad()
        if step_id %100 == 0:
            print("epoch {} step_id {} cost {} acc {}".format(epoch, step_id, cost, acc))